# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [17]:
df = data.groupby(['CustomerID', 'ProductName'])[['Quantity']].sum()
df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(data, index= ['ProductName'],columns= ['CustomerID'],values= 'Quantity', fill_value=0)
table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [14]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [31]:
similar_top5 = distances[33].sort_values(ascending=False)[1:6]
similar_top5

CustomerID
3909    0.095358
3531    0.093953
264     0.093953
2503    0.093498
3305    0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [32]:
top5_df = data[data['CustomerID'].isin(similar_top5.index)].sort_values('CustomerID')
top5_df.head(10)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
32311,264,Virgil,Small,1609342,240,Wine - Chablis 2003 Champs,1
34764,264,Virgil,Small,4356324,289,Sword Pick Asst,1
33544,264,Virgil,Small,2585657,361,Coffee Decaf Colombian,1
32037,264,Virgil,Small,6577594,297,Hinge W Undercut,1
29028,264,Virgil,Small,5538534,151,"Mushrooms - Black, Dried",1
29027,264,Virgil,Small,5275781,151,"Mushrooms - Black, Dried",1
28042,264,Virgil,Small,2009347,273,Bandage - Flexible Neon,1
27881,264,Virgil,Small,3883387,368,Flour - Teff,1
26104,264,Virgil,Small,5035449,158,Apricots Fresh,1
25659,264,Virgil,Small,3568777,269,Wine - Blue Nun Qualitatswein,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [33]:
top_products = top5_df.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index()
top_products.head()

,ProductName,Quantity
0,"Salsify, Organic",4
1,Quiche Assorted,3
2,Chocolate - Dark,3
3,Wine - Charddonnay Errazuriz,3
4,Bay Leaf,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [34]:
no_shop = top_products.merge(table, on='ProductName')
no_shop.head()

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,"Salsify, Organic",4,1,0,0,0,0,0,0,0,...,25,0,0,0,0,0,0,0,25,0
1,Quiche Assorted,3,1,1,1,0,0,0,0,0,...,25,0,0,0,0,0,25,0,0,0
2,Chocolate - Dark,3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
3,Wine - Charddonnay Errazuriz,3,0,0,0,0,0,0,1,0,...,0,25,0,0,0,0,0,25,0,0
4,Bay Leaf,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [45]:
customers = data['CustomerID'].unique()
recomendations = {}

for i in customers:
    users = pd.DataFrame(distances.loc[i].sort_values(ascending=False).head(6)).reset_index()
    similar = data[data['CustomerID'].isin(users['CustomerID'].unique())]
    prod_ranking = pd.DataFrame(similar.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)).reset_index()
    no_buy = set(prod_ranking['ProductName'].unique()) - set(similar[similar['CustomerID'] == i]['ProductName'].unique())
    recomendations[i] = list(prod_ranking[prod_ranking['ProductName'].isin(no_buy)].sort_values(by='Quantity', ascending=False).head()['ProductName'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [47]:
best_recommendations = pd.DataFrame.from_dict(recomendations, orient='index')
best_recommendations = best_recommendations.reset_index()
best_recommendations.columns = ['CustomerID', 
                     'Product 1', 
                     'Product 2', 
                     'Product 3', 
                     'Product 4', 
                     'Product 5']
best_recommendations.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,"Mushrooms - Black, Dried","Wine - Magnotta, Merlot Sr Vqa",Chicken - Soup Base,Wine - Chardonnay South,Milk - 1%
1,77352,Guinea Fowl,Grenadine,Ecolab - Mikroklene 4/4 L,"Oranges - Navel, 72","Coconut - Shredded, Sweet"
2,40094,"Water - Mineral, Natural",Pasta - Orecchiette,Quiche Assorted,"Oregano - Dry, Rubbed",Rabbit - Whole
3,23548,Banana Turning,Wanton Wrap,"Beef - Chuck, Boneless","Sole - Dover, Whole, Fresh",Apricots Fresh
4,78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Beef - Inside Round,Sprouts - Alfalfa


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.